In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
import time

In [2]:
all_data = []
all_limits = []

In [ ]:
#read in blob dataset

X_training = []
y_training = []
header = []
c = 0
with open("DataSets/blob_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([ float(x) for x in row])
        
with open("DataSets/blob_label.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(int(row[0]))


threshold = []
for i in range(len(X_training[0])):
    vector = [x[i] for x in X_training]
    threshold.append([min(vector),max(vector)])

all_limits.append(threshold)
        
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])

In [4]:
#read in iris dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/iris.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row[:-1]
            c+=1
            continue
        X_training.append([float(x) for x in row[:-1]])
        y_training.append(row[-1])

for i in range(len(y_training)):
    if(y_training[i] == "Setosa"):
        y_training[i] = 1
    if(y_training[i] == "Versicolor"):
        y_training[i] = 2
    if(y_training[i] == "Virginica"):
        y_training[i] = 3

threshold = []
for i in range(len(X_training[0])):
    vector = [x[i] for x in X_training]
    threshold.append([min(vector),max(vector)])

all_limits.append(threshold)


X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])


In [ ]:
#read in wine dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/winequality-white_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row[:-1]])
        y_training.append(float(row[-1]))
        #print(row)

threshold = []
for i in range(len(X_training[0])):
    vector = [x[i] for x in X_training]
    threshold.append([min(vector),max(vector)])

all_limits.append(threshold)


X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])



In [ ]:
#read in house price dataset
X_training = []
y_training = []
header = []
c = 0
with open("DataSets/house_att.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        if(c == 0):
            header = row
            c+=1
            continue
        X_training.append([float(x) for x in row])
        
with open("DataSets/house_price.csv","r") as file:
    r = csv.reader(file)
    for row in r:
        y_training.append(float(row[0]))


threshold = []
for i in range(len(X_training[0])):
    vector = [x[i] for x in X_training]
    threshold.append([min(vector),max(vector)])

all_limits.append(threshold)
        
print(len(X_training))
print(len(y_training))
X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.2) # 80% training and 20% testing data
all_data.append([X_train, X_test, y_train, y_test,header])





In [7]:
#Sort dataset by an attribute.(increasing order)
def sort_f(X,y,column):
    

  
    sortf = zip(X,y)
    sortf= sorted(sortf,key = lambda x:x[0][column])
    X = [x for x,y in sortf]
    y = [y for x,y in sortf]
    
    

    return X,y

In [8]:
class Tree:
    def __init__(self,X,feature_names,labels):
        
        self.X = X
        self.num_of_nodes = 0
        self.currentsplit = 0
        self.split_result = 0
        self.feature_names = feature_names #coloum names
        self.labels = labels#y
        self.nodes = []
        self.steps = [0]
        self.leaf = 0
        self.std = 0
        

In [9]:
class Node:
    def __init__(self,labels,X,feature_ids):
        
        self.split_result = 0
        self.split = 0 #which column / which feature id
        self.feature_ids = feature_ids
        self.labels = labels
        self.X = X
        self.nodes = []
        self.steps = []
        self.variance = []
        self.std = []
        self.thresholds = []
        
        self.leaf = 0 #true or false

In [10]:
def get_features(X,labels,column,bottom,top):
    
    #Return a portion of the dataset, which's attributes is in a certain


    sub_label = [labels[x] for x in range(len(labels)) if X[x][column]<top and X[x][column]>=bottom]
    sub_X = [X[x] for x in range(len(X)) if X[x][column]<top and X[x][column]>=bottom]

        
    return [sub_X,sub_label]

In [11]:
def calculate_std(data):
    
    #print(data)
    mean = sum(data)/len(data)
    
    variance = 0
    
    for i in data:
        variance += (i-mean)**2
    
    
    return math.sqrt(variance/len(data))
    

In [12]:
def calculate_dist(data1,data2):

    d1 = sum(data1)/len(data1)
    d2 = sum(data2)/len(data2)
    return abs(d2-d1)
    

In [13]:
def find_min_distance(sliced_data,column):
    
    dist = 0
    min_dist = -1
    index1 = -1
    index2 = -1
    
    #determine what to merge by the standard deviation
    
    for i in range(len(sliced_data)-1):

            dist = 0

            X = sliced_data[i][0]
            X2 = sliced_data[i+1][0]
            X = [ X[j][column] for j in range(len(X)) ] 
            X2 = [ X2[j][column] for j in range(len(X2)) ]
            
                
            #dist = abs(calculate_std(sliced_data[i][1])-calculate_std(sliced_data[i+1][1]))
            #dist = abs(calculate_std(X)-calculate_std(X2))
            dist = calculate_dist(X,X2)
            #dist = calculate_dist(sliced_data[i][1],sliced_data[i+1][1])


            #dist /=len(sliced_data[i][0][0]) 
            if(min_dist == -1):
                min_dist = dist
                index1 = i
                index2 = i+1
            if(dist<min_dist):
                min_dist = dist
                index1 = i
                index2 = i+1
    
    return index1,index2
    

In [14]:
def merge(sliced_data,column):
    
   

    index1,index2 = find_min_distance(sliced_data,column)
    

    sliced_data[index1][0] = sliced_data[index1][0]+sliced_data[index2][0]
    sliced_data[index1][1] = sliced_data[index1][1]+sliced_data[index2][1]

   
    
    sliced_data.pop(index2)

    return sliced_data

In [15]:
def pre_merge(sliced_data,index,std,column):
    
   

    #index1,index2 = find_min_distance(sliced_data,column)
    dist_local = 0

    for i in range(len(sliced_data[0][0][0])):

        X = sliced_data[index][0]
        X2 = sliced_data[index+1][0]
        #features = [ X[j][i] for j in range(len(X)) ] + [ X2[j][i] for j in range(len(X2)) ]
        #std_local += calculate_std(features)
        dist_local += calculate_dist([ X[j][i] for j in range(len(X)) ],[ X2[j][i] for j in range(len(X2)) ])



    if(dist_local <= std ) :

        sliced_data[index][0] = sliced_data[index][0]+sliced_data[index+1][0]
        sliced_data[index][1] = sliced_data[index][1]+sliced_data[index+1][1]
        sliced_data.pop(index+1)
        return sliced_data
   
    return sliced_data

In [16]:
def calculate_std_avg(features):
        dist= 0


        for i in range(len(features)-1):

            dist += calculate_dist(features[i],features[i+1])

        dist = dist/len(features)

        return dist

In [17]:
def get_features2(sliced_data,column):#return attribute vetctor for each cluster

    features = []

    for i in sliced_data:
        features.append([ i[0][j][column] for j in range(len(i[0])) ])
    
    return features

In [18]:
def preprocess(X,labels,column):#Az std valójában distance
    

    sliced_data = [[ [x] , [y] ] for x,y in zip(X,labels)]

    

    #features = [ [X[j][column]] for j in range(len(X)) ] 
    labels = [ [j] for j in labels ] 


    std = 0

    for i in range(len(sliced_data[0][0][0])):
        std += calculate_std_avg(get_features2(sliced_data,i))


    index = -1
    l = 0

    while(len(sliced_data)>=20):

        index += 1
        if(index>=len(sliced_data)-1):
            index = 0
            if(len(sliced_data) == l):
                break
            else:
                l = len(sliced_data)
            std = 0
            for i in range(len(sliced_data[0][0][0])):
                std += calculate_std_avg(get_features2(sliced_data,i))
            
            

        sliced_data = pre_merge(sliced_data,index,std,column)
        

    return sliced_data

In [19]:
def find_best_split(X,labels,feature_ids,steps,div):
    
    step = -1

    features = 0 
    svalue = 0
    
    min_error = -1
    min_sliced_data = []
    min_regressions = []
    min_steps = -1
    min_column = -1



    for column in feature_ids:
            
            
            X,labels = sort_f(X,labels,column)        

            if(steps[column] > 8 ):
                    continue


            sliced_data = preprocess(X,labels,column)
           
            while(len(sliced_data)>div):

                sliced_data = merge(sliced_data,column)

            
            error = 0
            for i in range(len(sliced_data)):
                
                error += calculate_std(sliced_data[i][1])
           
            if(min_error == -1):
                min_sliced_data = sliced_data
                
                min_steps = step
                min_column = column
                min_error = error
                continue
            #choose the splited data with the minimum error
            if(error<min_error):
                min_sliced_data = sliced_data
                min_error = error
                min_steps = step
                min_column = column
    
    #return split,split_value
    steps[min_column] += 1
    
    return min_sliced_data,min_column,steps


In [20]:
def build_tree(X,feature_ids,labels,leaf_size,limit,div,depth,steps,X_train,y_train):
    
    #find the best split for sub_nodes
    #return sub_nodes
    split = 0 
    sub_nodes = []
    
    st = steps.copy()

    #get the best split 
    split_data,column,st = find_best_split(X,labels,feature_ids,st,div)

    
    #create objects and inicialize them
    for i in split_data:
        
        node = Node(i[1],i[0],feature_ids)
        node.split = column 

        node.split_result = i[0][-1][column]
        node.depth = depth
        
        node.top =   max([x[column] for x in node.X ])
        node.bottom = min([x[column] for x in node.X ])

        for j in range(len(node.X[0])):
            node.thresholds.append([min([x[j] for x in node.X ]),max([x[j] for x in node.X ])])

        node.steps = st

        node.std.append(calculate_std(node.labels))

        
        if(len(node.labels) <= leaf_size or calculate_std(node.labels)<limit):
            node.leaf = 1

            
        else:
            node.leaf = 0
            
        sub_nodes.append(node)
    
    if(depth == 2000):
        for i in sub_nodes:
            i.leaf = 1
        return sub_nodes
    
    depth +=1

    leaf =  0
    for i in sub_nodes:
        if(i.leaf == 1):
            leaf +=1
            
    if(leaf == len(sub_nodes)):
        for i in sub_nodes:

            if(calculate_std(i.labels)>=100):

                print(calculate_std(i.labels),len(i.labels))

            
        return sub_nodes
  
    else:
        for node in sub_nodes:

            if(node.leaf == 0):

                node.nodes = build_tree(node.X,node.feature_ids,node.labels,leaf_size,limit,div,depth,node.steps,X_train,y_train)


    return sub_nodes   

In [21]:
def inicialize(X,feature_names,labels,leaf_size,limit,div):
    
    depth = 1
    feature_ids = [x for x in range(len(feature_names))]
    tree = Tree(X,feature_names,labels)
    
    steps = []
    for i in feature_ids:
        steps.append(0)
    #start building tree
    tree.nodes = build_tree(X,feature_ids,labels,leaf_size,limit,div,depth,steps,X,labels)
    return tree

In [22]:
forest = []
times = []

divide = [2,3,4,5]
counter = 0

for i in all_data:
    f = []
    t = []


    if(counter == 0):
        size =20
        var = 0.5
    if(counter == 1):
        size =5
        var = 0.2
    if(counter == 2):
        size =20
        var = 0.5
    if(counter == 3):
        size = 20
        var = 1


    for j in divide:
       
        print("-------------------------------------------------------")


        start = time.time()
        tree = inicialize(i[0],i[-1],i[2],size,var,j)
        end = time.time()
        f.append(tree)
        print(end-start)
        t.append(end-start)
        
    counter+=1
    forest.append(f)
    times.append(t)
    

-------------------------------------------------------


2.2938802242279053
-------------------------------------------------------
1.4639387130737305
-------------------------------------------------------
1.486908197402954
-------------------------------------------------------
1.440079689025879
-------------------------------------------------------
0.014581441879272461
-------------------------------------------------------
0.009378910064697266
-------------------------------------------------------
0.0060575008392333984
-------------------------------------------------------
0.005351543426513672
-------------------------------------------------------
25.388563871383667
-------------------------------------------------------
12.848352670669556
-------------------------------------------------------
7.490605354309082
-------------------------------------------------------
6.552295446395874
-------------------------------------------------------
4.672508001327515
-------------------------------------------------------
3.2896182537078857
--

In [23]:
def predict(node,value,thresholds,weight):


    

    if(node.leaf == 1 ):


        p = sum(node.labels)/len(node.labels)

        X = []
        #error = 0

        for i in range(len(node.X[0])):

         
            X.append(sum([ ((node.X[j][i]-thresholds[i][0])/(thresholds[i][1]-thresholds[i][0])) * weight[i]  for j in range(len(node.X)) ]) )
    
        X = [x / len(node.X) for x in X]
        
        #print(X,p)
        error = 0

        for i in range(len(X)):

            error +=  abs(X[i]- ((value[i]-thresholds[i][0])/(thresholds[i][1]-thresholds[i][0])) * weight[i] )
        

        p=[p,error]
        
        return p
    else:
        
        c=0
        
        p = 0
        
        values = []
        for i in node.nodes:

            if (i.top>=value[i.split] and i.bottom<=value[i.split]  ):

                c+=1
                
                p = predict(i,value,thresholds,weight)

                for k in p:
                    values.append(k)
        if(c == 0  ):
           

            p = sum(node.labels)/len(node.labels)

            X = []
            #error = 0
            for i in range(len(node.X[0])):
                X.append(sum([ ((node.X[j][i]-thresholds[i][0])/(thresholds[i][1]-thresholds[i][0])) * weight[i]  for j in range(len(node.X)) ]) )


            X = [x /len(node.X) for x in X]
    
            error = 0
        
            for i in range(len(X)):
                error +=  abs(X[i]- ((value[i]-thresholds[i][0])/(thresholds[i][1]-thresholds[i][0])) * weight[i] )

            
            p=[p,error]

            return p

        return values
        

In [24]:
def get_threshold(node):

    if(node.leaf == 1):
        return node.steps,node.depth


    thresholds = [0] * len(node.X[0])
    
    for i in node.nodes:
        s,depth = get_threshold(i)

        for j in range(len(s)):
            thresholds[j] += s[j]
    return thresholds,depth
    

    

In [28]:
print(times)
predicted_value = []#predicted points
difference = []
MSE = []
sum_y = []

#all_difference = []
all_mse = []
all_std = []
#for each tree we calculate the mean squared error



for v in range(len(forest)):
    MSE = []
    counter = 0
    for i in forest[v]:
        counter += 1
        j = all_data[v]
        
        X_test = j[1]
        y_test = j[3]
        pv = []
        error = 0
        mse = 0
        l = 0
        thresholds,depth = get_threshold(i)
        thresholds = [x/depth for x in thresholds]
        
        for k in range(len(X_test)): 


            pr = predict(i,X_test[k],all_limits[v],thresholds)

            p = 0

            prev = 1000000000000
            if(pr[0] != 0):

                for f in range(0,len(pr)-1,2):
                    if(pr[f+1]<prev):
                        prev = pr[f+1]
                        p = pr[f]


            error += abs(y_test[k]-p)

            mse += (y_test[k]-p)**2
            pv.append(p)
           


        sum_y.append(sum(y_test))
        predicted_value.append(pv)
        difference.append(error)
        MSE.append(mse/len(X_test))
        #MSE.append(mse/l)

    all_mse.append(MSE)
    
    
print(difference)
print(sum_y)

print(all_mse)
'''with open("Results/difference_rtn.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    for row in all_mse:
        writer.writerow(row)
        
with open("Results/times_rtn.csv", 'a',newline='') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a row to the csv file
    
    for row in times:
        
        writer.writerow(row)'''

print(len(predicted_value[0]))


[[2.2938802242279053, 1.4639387130737305, 1.486908197402954, 1.440079689025879], [0.014581441879272461, 0.009378910064697266, 0.0060575008392333984, 0.005351543426513672], [25.388563871383667, 12.848352670669556, 7.490605354309082, 6.552295446395874], [4.672508001327515, 3.2896182537078857, 2.5632381439208984, 2.0396902561187744]]
[237.92263276218355, 514.1752805275729, 97.05599282080658, 209.05028947429543, 2.3214285714285716, 1.688888888888889, 2.743650793650793, 2.743650793650793, 522.6243941362828, 534.293436061541, 534.1472034601878, 595.0155008910308, 5725.147641982945, 5975.497395587674, 6312.75636201757, 6460.951125094305]
[4019, 4019, 4019, 4019, 60, 60, 60, 60, 5707.0, 5707.0, 5707.0, 5707.0, 620464.5291393183, 620464.5291393183, 620464.5291393183, 620464.5291393183]
[[0.08390730799113708, 0.13911573842969793, 0.036498573305620396, 0.06301990647254917], [0.07011054421768707, 0.03845267489711934, 0.06920569832871423, 0.06920569832871423], [0.48393053658682395, 0.53681959239036